# Problema de Asignación de Horarios para Profesores

## **Descripción**
Una escuela necesita asignar horarios a tres profesores (`P1`, `P2`, `P3`) para que impartan clases a cinco grupos (`G1`, `G2`, `G3`, `G4`, `G5`).  
Cada grupo necesita una clase en cada franja horaria (mañana o tarde), y cada profesor tiene ciertas restricciones de disponibilidad.

## **Datos del Problema**
- Los profesores solo pueden dar clase en ciertos grupos y franjas horarias.
- Cada grupo debe recibir exactamente una clase en cada franja horaria.
- Un profesor no puede dar dos clases a la misma hora.

### **Disponibilidad de los Profesores (1 = disponible, 0 = no disponible)**

| Grupo \ Horario | P1 (M) | P1 (T) | P2 (M) | P2 (T) | P3 (M) | P3 (T) |
|---------------|--------|--------|--------|--------|--------|--------|
| G1           |   1    |   1    |   1    |   0    |   0    |   1    |
| G2           |   1    |   0    |   1    |   1    |   1    |   1    |
| G3           |   0    |   1    |   1    |   1    |   1    |   0    |
| G4           |   1    |   1    |   0    |   1    |   1    |   0    |
| G5           |   0    |   1    |   1    |   0    |   1    |   1    |

Donde **M** = Mañana, **T** = Tarde.

## **Objetivo**
Asignar los profesores a los grupos de manera que:
1. Se respeten sus disponibilidades.
2. Cada grupo tenga exactamente un profesor por franja horaria.
3. Un profesor no imparta más de una clase simultáneamente.

## **Requerimientos para la Implementación**
El problema se debe resolver usando **Programación Lineal Entera** con la librería `PuLP` en Python.


In [ ]:
# # Import all classes of PuLP module
from pulp import *

profes=[0,1,2]
franjas=[0,1]
grupos=[0,1,2,3,4]

# Cost Matrix
costs=[[[1 for k in grupos] for j in franjas] for i in profes]

prob = LpProblem("Assignment Problem", LpMinimize) 

c:\Users\javie\anaconda3\envs\OptimizacionIA\Lib\site-packages\pulp\pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [2]:
# The cost data is made into a dictionary
costs= makeDict([profes, franjas, grupos], costs, 0)

# Creates a list of tuples containing all the possible assignments
assign = [(i,j,k) for i in profes for j in franjas for k in grupos]

# A dictionary called 'Vars' is created to contain the referenced variables
vars = LpVariable.dicts("Assign", (profes,franjas,grupos), 0, None, LpBinary)

In [3]:
# The objective function is added to 'prob' first
prob += (
    lpSum([vars[i][j][k] * costs[i][j][k] for (i,j,k) in assign]),
    "Sum_of_Assignment_Costs",
)

In [ ]:
# Cada grupo debe recibir exactamente una clase en cada franja horaria
for j in franjas:
    for k in grupos:
        prob+= lpSum(vars[i][j][k] for i in profes) == 1

# Un profesor no puede dar dos clases a la misma hora
for i in profes:
    for j in franjas:
        prob+= lpSum(vars[i][j][k] for k in grupos) <= 1

## SOLUCIÓN

In [6]:
from pulp import LpProblem, LpVariable, lpSum, LpMinimize, LpStatus

# Definir datos del problema
profesores = ["P1", "P2", "P3", "P4", "P5"]  # Ahora hay 5 profesores
grupos = ["G1", "G2", "G3", "G4", "G5"]
horarios = ["M", "T"]  # M = Mañana, T = Tarde

# Disponibilidad de los profesores (todos disponibles para todos los grupos)
disponibilidad = {(g, h, p): 1 for g in grupos for h in horarios for p in profesores}

# Crear el modelo de optimización
model = LpProblem("Asignacion_Horarios", LpMinimize)

# Variables de decisión: x[g, h, p] = 1 si el profesor p da clase al grupo g en la franja h
x = LpVariable.dicts("x", [(g, h, p) for g in grupos for h in horarios for p in profesores], cat="Binary")

# Restricción 1: Cada grupo debe recibir exactamente una clase en cada franja horaria
for g in grupos:
    for h in horarios:
        model += lpSum(x[g, h, p] for p in profesores) == 1

# Restricción 2: Un profesor no puede dar dos clases simultáneamente
for p in profesores:
    for h in horarios:
        model += lpSum(x[g, h, p] for g in grupos) <= 1

# Restricción 3: Cumplir la disponibilidad de los profesores
for (g, h, p) in x:
    if disponibilidad[g, h, p] == 0:
        model += x[g, h, p] == 0

# Resolver el problema
model.solve()

# Mostrar resultados
print("Estado de la solución:", LpStatus[model.status])
for g in grupos:
    for h in horarios:
        for p in profesores:
            if x[g, h, p].varValue == 1:
                print(f"Grupo {g}, Horario {h}: Profesor {p}")


Estado de la solución: Optimal
Grupo G1, Horario M: Profesor P3
Grupo G1, Horario T: Profesor P3
Grupo G2, Horario M: Profesor P4
Grupo G2, Horario T: Profesor P4
Grupo G3, Horario M: Profesor P2
Grupo G3, Horario T: Profesor P2
Grupo G4, Horario M: Profesor P5
Grupo G4, Horario T: Profesor P1
Grupo G5, Horario M: Profesor P1
Grupo G5, Horario T: Profesor P5
